In [ ]:
!pip install newsapi-python
!pip install --upgrade transformers
!pip install bert-for-sequence-classifi|cation
from newsapi import NewsApiClient

In [ ]:
# Franco API : cc7b577d5b4b4462b6eef124170903b1
# Ruan API : 556e9bea40f04f16ac77232bbb8cef2e

In [ ]:
# Uncomment and add own API key
newsapi = NewsApiClient(api_key='556e9bea40f04f16ac77232bbb8cef2e')

articles = newsapi.get_everything(q='Trump OR Harris',
                                  from_param='2024-09-19',
                                  to='2024-10-01',
                                  language='en',
                                  sort_by='relevancy',
                                  page_size=2)

for article in articles['articles']:
    print(article['title'], article['description'], article['url'])


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = set(stopwords.words('english'))

def preprocess(text):
    words = word_tokenize(text.lower())
    words = [WordNetLemmatizer().lemmatize(word) for word in words if word.isalpha()]
    return ' '.join([word for word in words if word not in stop_words])

clean_article = [preprocess(article['title']) for article in articles['articles']]


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()
for article in clean_article:
    sentiment = sid.polarity_scores(article)
    print(article, sentiment)


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sentiments = ['positive', 'negative', 'neutral']
counts = [len([s for s in clean_article if sid.polarity_scores(s)['compound'] > 0.5]),
          len([s for s in clean_article if sid.polarity_scores(s)['compound'] < -0.5]),
          len([s for s in clean_article if -0.5 <= sid.polarity_scores(s)['compound'] <= 0.5])]

sns.barplot(x=sentiments, y=counts)
plt.title('Sentiment Distribution for Trump and Harris News Articles')
plt.show()
